In [72]:
import ray
from typing import Dict, List
import numpy as np
import pyarrow.json as pajson
block_size = 10 << 25 # Set block size to 300MB
ds = ray.data.read_json('/home/userroot/dev/jiucai/crawl/stock_report/survey_contents'
,read_options=pajson.ReadOptions(block_size=block_size))
#ds = ds.limit(100)


In [21]:
block_size = 10 << 25
block_size

335544320

In [27]:
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoModel
from llama_index.core.node_parser import  SentenceSplitter #SimpleNodeParser
from llama_index.core.schema import Node
from llama_index.core.schema import Document
import pandas as pd
import pyarrow as pa
import warnings
import pandas as pd
import copy
from llama_index.core.schema import TextNode, NodeRelationship, RelatedNodeInfo

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)


class EmbedNodes:
    def __init__(self):
        import os
        #os.environ['HTTPS_PROXY'] = 'http://100.109.83.118:808/'
        self.embedding_model = HuggingFaceEmbeddings(model_name='/data/models/bge-large-zh')

    #列存储的方式
    def __call__(self, data_batch: Dict[str, np.ndarray]) -> Dict[str, List[Node]]:
        nodes = []
        summaries = []
        answers_batch = data_batch['result']
        for a in answers_batch:
            for aa in a['data']:

                #t = aa['CONTENT'] 
                node = TextNode(text=str(aa))
                nodes.append(node)
                summaries.append(str(aa))
                break
        
        embeddings = self.embedding_model.embed_documents(summaries)
        assert len(nodes) == len(embeddings)

        for node, embedding in zip(nodes, embeddings):
            node.embedding = embedding
        return {"embedded_nodes": nodes}

In [73]:
class EmbedNodes:
    def __init__(self):
        import os
        #os.environ['HTTPS_PROXY'] = 'http://100.109.83.118:808/'
        self.embedding_model = HuggingFaceEmbeddings(model_name='/data/models/bge-large-zh')

    #列存储的方式
    def __call__(self, data_batch: Dict[str, np.ndarray]) -> Dict[str, List[Node]]:
        nodes = []
        summaries = []
        answers_batch = data_batch['result']
        for a in answers_batch:
            for aa in a['data']:

                #t = aa['CONTENT'] 
                text_list =  [str(value) for value in aa.values()]
                joined_text = ','.join(text_list)
                node = TextNode(text=str(joined_text))
                nodes.append(node)
                summaries.append(str(joined_text))
                break
        
        embeddings = self.embedding_model.embed_documents(summaries)
        assert len(nodes) == len(embeddings)

        for node, embedding in zip(nodes, embeddings):
            node.embedding = embedding
        return {"embedded_nodes": nodes}

In [74]:
embeds = ds.map_batches(EmbedNodes, concurrency=4, num_gpus=1, batch_size=400)
    

In [29]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = Ollama(model="qwen:7b", request_timeout=120.0)
Settings.embed_model = HuggingFaceEmbedding(
    model_name="/data/models/bge-large-zh"
)

In [75]:
ray.data.DataContext.get_current().execution_options.verbose_progress = True
stock_docs_nodes = []
title_dict = {}
for row in embeds.iter_rows():
    node:Node = row["embedded_nodes"]
    if not node.id_ in title_dict:
        title_dict[node.id_] = True
        assert node.embedding is not None
        stock_docs_nodes.append(node)

2024-03-12 15:27:53,280	INFO set_read_parallelism.py:115 -- Using autodetected parallelism=200 for stage ReadJSON to satisfy DataContext.get_current().min_parallelism=200.
2024-03-12 15:27:53,281	INFO set_read_parallelism.py:122 -- To satisfy the requested parallelism of 200, each read task output is split into 8 smaller blocks.
2024-03-12 15:27:53,286	INFO streaming_executor.py:112 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadJSON] -> ActorPoolMapOperator[MapBatches(EmbedNodes)]
2024-03-12 15:27:53,288	INFO streaming_executor.py:113 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), exclude_resources=ExecutionResources(cpu=0, gpu=0, object_store_memory=0), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2024-03-12 15:27:53,290	INFO streaming_executor.py:115 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().exec

2024-03-12 15:27:53,502	INFO actor_pool_map_operator.py:114 -- MapBatches(EmbedNodes): Waiting for 4 pool actors to start...


Running 0:   0%|          | 0/25 [00:00<?, ?it/s]

In [76]:
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from chromadb.errors import DuplicateIDError
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = Ollama(model="deepseek-llm:67b", request_timeout=120.0)
Settings.embed_model = HuggingFaceEmbedding(
    model_name="/data/models/bge-large-zh"
)

# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./stock_db")

# create collection
chroma_collection = db.get_or_create_collection("qa_base")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# index = VectorStoreIndex.from_vector_store(
#     vector_store
# )
#create your index
try:
    index = VectorStoreIndex(
        nodes=stock_docs_nodes, storage_context=storage_context
    )
except DuplicateIDError:
    print("duplicat id, ignore it!")

# # create a query engine and query
# query_engine = index.as_query_engine()
# response = query_engine.query("What is the meaning of life?")
# print(response)

OperationalError: attempt to write a readonly database

In [55]:
# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [62]:
Settings.llm = Ollama(model="deepseek-llm:67b", request_timeout=600.0)
Settings.embed_model = HuggingFaceEmbedding(
    model_name="/data/models/bge-large-zh"
)
query_engine = index.as_query_engine(
    similarity_top_k = 2
)
#response = query_engine.query("详细说明一下ADC药物的过程，相关企业有哪些？")

In [63]:
from IPython.display import Markdown, display
response = query_engine.query("比亚迪独特的技术优势和企业优势有哪些？用中文回答")
display(Markdown(f"<b>{response}</b>"))

<b>根据提供的信息，比亚迪拥有一系列独特的技术优势和竞争优势。其中包括：

1. **海豹高电压电驱升压充电方案**：这项技术实现了宽域恒功率充电，使得电动车能够在15分钟内行驶300公里，大幅减少了用户等待时间。

2. **云辇智能车身控制系统**：这是全球首个新能源专属的智能车身控制技术，能够系统化考量新能源汽车的垂向控制问题，以高智能、护安全、稳驾乘、全覆盖的技术优势，为用户带来兼顾舒适性与操控性的极致驾乘体验。

3. **DMO专业越野平台**：这是一个以插混为核心的平台，通过比亚迪全新混动非承载式架构和越野专用混动架构的极致融合，充分发挥比亚迪在高性能底盘、智能电四驱及越野专用动力总成等方面的领先优势，成就超级整车安全之上超强动力、强悍越野与极致能耗之间的最佳平衡。

4. **多品牌梯度布局**：依托于本集团核心技术的持续迭代与创新，比亚迪乘用车业务逐步形成由比亚迪品牌、腾势品牌、仰望品牌及方程豹品牌所构建的多品牌梯度布局，覆盖从家用到豪华、从大众到个性化，全面满足用户多方位全场景的用车需求。

5. **新能源公交电动化转型**：在公共交通领域，比亚迪领先的三电技术和设计水平能够为城市提供高效的公交运营效率和舒适的乘客体验。同时，比亚迪也在积极推动全球新能源汽车的发展，例如巴西市场的海豚车型就取得了显著的销售成绩。

这些技术优势和企业战略布局共同构成了比亚迪的核心竞争力，使其能够在激烈的市场竞争中脱颖而出。</b>

In [64]:
response = query_engine.query("ADC药物的独特优势，和目前成功的产品有哪些？用中文回答")
display(Markdown(f"<b>{response}</b>"))

<b>ADC药物是一种结合了单克隆抗体特异性和细胞毒性化合物杀伤力的新型治疗方法。相比传统化疗药物，它具有更高的靶向性和药效比，能够减少对正常组织的副作用。此外，它的独特结构和作用机制还可以克服肿瘤耐药性的问题，提高临床治疗的疗效。

目前市场上已经成功上市的ADC药物包括Adcetris、Kadcyla、Mylotarg和Besponsa等。这些药物已经在血液瘤和实体瘤的治疗中取得了显著的疗效，并且正在被进一步开发用于其他适应症的治疗。</b>

In [67]:
response = query_engine.query("国内ADC药物的生产企业有哪些？有哪些独有的技术或者优势？用中文回答")
display(Markdown(f"<b>{response}</b>"))

<b>根据给定的上下文信息，中国国内的ADC（抗体偶联）药物生产企业有新诺威和迈威生物。其中，巨石生物是新诺威的子公司，其EGFR-ADC产品在中低剂量组已经观察到肿瘤缩小的疗效反应；而9MW2821则是迈威生物自主研发的Nectin-4 ADC，具有更均一的组分、更加稳定的结构和更加优异的肿瘤递送能力。此外，新诺威还在疫苗平台方面有所布局，计划开发RSVmRNA疫苗、VZVmRNA疫苗和HPV治疗性疫苗等产品。</b>

In [57]:
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [53]:
response

Response(response='抱歉，我之前的回答似乎有误。以下是针对吉利汽车独特技术优势和企业优势的正确解答：\n\n1. **技术优势**：\n   - 精准匹配消费需求：吉利汽车在研发过程中注重市场调研，精准把握消费者需求，从而提升产品技术含量。\n   - 高效节能技术应用：随着全球能源形势变化，高效节能成为汽车行业的重要发展方向。吉利汽车在这方面积累了丰富经验和技术成果。\n   \n2. **企业优势**：\n   - 产业链完整：吉利集团在汽车产业内拥有包括研发、设计、生产、销售等在内的完整产业链，为旗下汽车公司提供了强大的技术支撑和资源保障。\n   - 国际化布局：吉利汽车在国内外市场均有布局，这既体现了其全球化战略视野，也意味着其在全球范围内具有较强的竞争力和技术优势。\n   \n综上所述，吉利汽车凭借精准市场需求、高效节能技术应用以及产业链完整等优势，在汽车行业内展现出强大的竞争力和持续的技术创新能力。\n', source_nodes=[NodeWithScore(node=TextNode(id_='cc0bb39c-f05c-47a5-81e0-f592486d0bc7', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="{'SECUCODE': '300698.SZ', 'SECURITY_CODE': '300698', 'SECURITY_NAME_ABBR': '万马科技', 'NOTICE_DATE': datetime.datetime(2023, 10, 26, 0, 0), 'RECEIVE_START_DATE': datetime.datetime(2023, 10, 26, 0, 0), 'RECEIVE_END_DATE': datetime.datetime(2023, 10, 26, 0, 0), 'RECEIVE_OBJECT': '东北证券', 'RECEIVE_PLACE': '公司会议室', 'RECEIVE_WAY_EXPLAIN': '特定对象调研', 'INVESTIGATORS': '吴肖寅,唐凯,刘云坤